<a href="https://colab.research.google.com/github/vcutler-evr/vcutler-evr/blob/main/environmental_contamination_updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# for future work:
# think of a plan for re-geocoding or deleting the incorrectly geocoded addresses

# Code

## Install or Load Libraries

In [1]:
!pip install fused
!pip install arcgis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.1/354.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 941.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import fused
import arcgis
import getpass
import pandas as pd
import geopandas as gpd
from arcgis.gis import GIS
from shapely.geometry import Point

# for reading in files directly from an online website
import requests
import zipfile
import io
import csv

import numpy as np
import pyproj

import time

## Manual Authentication

In [3]:
gis = GIS(
  url="https://www.arcgis.com",
  username="victoria_evrealty",
  password="_FD8jdxuPYCHFmD"
)

## Display Options

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 400) # 400 characters

## Functions

In [5]:
# transform to a geopandas dataframe
def arcgis_point_to_shapely(point):
    return Point(point.x, point.y)

In [6]:
def getOnlineFile(url_of_interest, file_of_interest_name, file_of_interest_type, file_name_prefix):
    """
    Processes data based on the provided URL, file of interest, and file name prefix.

    Args:
    url_of_interest (str): The URL where data is located or relevant.
    file_of_interest (str): The specific file to process or analyze.
    file_name_prefix (str): The prefix to append to output files or results.

    Returns:
    None: The function may output files or results based on the input parameters.
    """
    # url of the online zipped data download
    url = url_of_interest

    # headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }

    try:
        response = requests.get(url, headers=headers, allow_redirects=True)

        # status code of the response
        print('HTTP Status Code:', response.status_code)

        # if the status code is 200 (OK): process the response
        if response.status_code == 200:
            # Use BytesIO to treat bytes as a file
            zip_file = io.BytesIO(response.content)

            # Use zipfile to read the ZIP content
            with zipfile.ZipFile(zip_file, 'r') as archive:
                # list all files in the zipped folder
                files_in_zip = archive.namelist()
                #print('Files in the zip:', files_in_zip)

                # target file string to look for
                target_file = file_of_interest_name + file_of_interest_type

                # create the output file name string
                output_file_name = file_name_prefix + "_" + file_of_interest_name

                # check if the target file string is in the zip folder
                if target_file in archive.namelist():
                    # read the target file and save in the global environment as the output file name string assigned above
                    with archive.open(target_file) as file:
                        globals()[output_file_name] = pd.read_csv(file, delimiter='\t', encoding='ISO-8859-1', on_bad_lines='skip') # skipping bad lines, will want to fix this in the future as noted in the future work section above

                        # print the head
                        #print(df.head())

                else:
                    print("File not found in the zip folder")
        else:
            # Print an error message if the status is not 200
            print("Failed to download the file, status code:", response.status_code)
    except Exception as e:
        # Handle and print any errors that occur during the request or processing
        print("An error occurred:", str(e))

    # other tests/checks:
    # print("Processing data from:", url_of_interest)
    # print("File of interest name:", file_of_interest_name)
    # print("File of interest type:", file_of_interest_type)
    # print("Filename prefix:", file_name_prefix)
    # print("Filename:", output_file_name)

# Example usage:
#getOnlineFile(url_of_interest = "https://www.envirostor.dtsc.ca.gov/public/hwmpexport/EnviroStorHazardousWaste.zip", file_of_interest_name = "sites", file_of_interest_type = ".txt", file_name_prefix = "hw")

## EnviroStor: Clean Up Sites

In [ ]:
# # envirostor: clean up sites
# getOnlineFile(url_of_interest = "https://www.envirostor.dtsc.ca.gov/public/export/EnviroStorCleanupSites.zip", file_of_interest_name = "sites", file_of_interest_type = ".txt", file_name_prefix = "envirostor_cs")

# envirostor_cs_sites_linked = envirostor_cs_sites
# envirostor_cs_sites_linked['ENVIROSTOR_ID'] = envirostor_cs_sites_linked['ENVIROSTOR_ID'].astype(str)
# envirostor_cs_sites_linked['case_details'] = "https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=" + envirostor_cs_sites["ENVIROSTOR_ID"]
# envirostor_cs_sites_linked.head()
# envirostor_cs_sites_linked_nonas = envirostor_cs_sites_linked.dropna(subset=['LATITUDE', 'LONGITUDE'])
# print(envirostor_cs_sites_linked.shape)
# print(envirostor_cs_sites_linked_nonas.shape)
# envirostor_cs_sites_linked_nonas_gdf = gpd.GeoDataFrame(
#     envirostor_cs_sites_linked_nonas,
#     geometry=[Point(xy) for xy in zip(envirostor_cs_sites_linked_nonas['LONGITUDE'], envirostor_cs_sites_linked_nonas['LATITUDE'])],
#     crs="EPSG:4326"
# )
# envirostor_cs_sites_linked_nonas_gdf.head()
# # remove extraneous columns: CUF_CASE is clean up fund case - not sure if this is necessary to keep in. RB_CASE_NUMBER is the water board case number
# envirostor_cs_sites_clean = envirostor_cs_sites_linked_nonas_gdf[['PROJECT_NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'COUNTY', 'SITE_TYPE', 'SITE_TYPE_DETAILED', 'REGULATORY_AGENCIES_INVOLVED', 'ENVIROSTOR_ID', 'SPECIAL_PROGRAM', 'STATUS', 'STATUS_DATE', 'PAST_USES', 'POTENTIAL_MEDIA_AFFECTED', 'RESTRICTED_USE', 'SITE_MGMT_REQ', 'FUNDING', 'case_details', 'geometry']]
# print(envirostor_cs_sites_clean.shape)
# envirostor_cs_sites_clean.head()

HTTP Status Code: 200
(13651, 34)
(13645, 34)
(13645, 20)


,PROJECT_NAME,ADDRESS,CITY,STATE,ZIP,COUNTY,SITE_TYPE,SITE_TYPE_DETAILED,REGULATORY_AGENCIES_INVOLVED,ENVIROSTOR_ID,SPECIAL_PROGRAM,STATUS,STATUS_DATE,PAST_USES,POTENTIAL_MEDIA_AFFECTED,RESTRICTED_USE,SITE_MGMT_REQ,FUNDING,case_details,geometry
0,HARRIS DRY CLEANERS,2801 MARTIN LUTHER KING JR. WAY,OAKLAND,CA,94609,ALAMEDA,State Response,State Response or NPL,"SMBRP, RWQCB 2 - San Francisco Bay, US EPA",1720109,NaN,Active,2000-06-16 00:00:00,"DRY CLEANING, NONE, NONE, NONE, NONE, NONE","OTH, SOIL, SV, IA, IA, IA, IA, IA",NO,NONE SPECIFIED,Orphan Funds,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=1720109,POINT (-122.27167 37.81813)
1,LOCKHEED PROPULSION-BEAUMONT NO. 2,JACK RABBIT TRAIL,BEAUMONT,CA,92223,RIVERSIDE,State Response,State Response or NPL,SMBRP,33370038,NaN,Active,2006-12-15 00:00:00,AEROSPACE ROCKET TESTING/LAUNCH,"OTH, SED, SOIL, SURFW",NO,NONE SPECIFIED,Responsible Party,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=33370038,POINT (-117.03111 33.93000)
2,Taylor Road,Taylor Road,Penryn,CA,95663,PLACER,Voluntary Cleanup,Voluntary Agreement,SMBRP,60002241,NaN,Active,2024-03-04 00:00:00,"AGRICULTURAL - ORCHARD, AGRICULTURAL - ORCHARD","SOIL, SOIL",NO,NONE SPECIFIED,Responsible Party,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=60002241,POINT (-121.15722 38.85810)
3,Country Fair Shopping Center,12051 & 12075 Central Avenue,Chino,CA,91710,SAN BERNARDINO,Voluntary Cleanup,Voluntary Agreement,SMBRP,60002479,NaN,Active,2017-01-31 00:00:00,DRY CLEANING,"SOIL, SV",NO,NONE SPECIFIED,Responsible Party,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=60002479,POINT (-117.68795 34.03453)
4,GBF / PITTSBURG DUMPS,SOMERVILLE RD & JAMES DONLON BLVD,ANTIOCH,CA,94509,CONTRA COSTA,Federal Superfund,State Response or NPL,"SMBRP, IWMB, RWQCB 5S - Central Valley, US EPA, CONTRA COSTA COUNTY",7490038,NaN,Active,1988-01-01 00:00:00,"LANDFILL - CONSTRUCTION, LANDFILL - DOMESTIC, LANDFILL - HAZARDOUS WASTE","OTH, SOIL",YES,"REM, DAY, HOS, LUC, NOWN, NDAM, NSUB, EXT, SCH, FOOD, RES",Responsible Party,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=7490038,POINT (-121.84686 37.98815)


In [ ]:
# # envirostor clean up sites
# #loi = gis.content.get('49bb85274e434c5396daeff04379f315') # envirostor clean up sites id

# # get data in correct format
# transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

# new_features = []

# for index, row in envirostor_cs_sites_clean.iterrows():
#     # Convert the coordinates
#     x, y = transformer.transform(row.geometry.x, row.geometry.y)
#     attributes = {
#         'PROJECT_NAME': row['PROJECT_NAME'],
#         'ADDRESS': row['ADDRESS'],
#         'CITY': row['CITY'],
#         'STATE': row['STATE'],
#         'ZIP': row['ZIP'],
#         'COUNTY': row['COUNTY'],
#         'SITE_TYPE': row['SITE_TYPE'],
#         'SITE_TYPE_DETAILED': row['SITE_TYPE_DETAILED'],
#         'REGULATORY_AGENCIES_INVOLVED': row['REGULATORY_AGENCIES_INVOLVED'],
#         'ENVIROSTOR_ID': row['ENVIROSTOR_ID'],
#         'STATUS': row['STATUS'],
#         'STATUS_DATE': row['STATUS_DATE'],
#         'PAST_USES': row['PAST_USES'],
#         'POTENTIAL_MEDIA_AFFECTED': row['POTENTIAL_MEDIA_AFFECTED'],
#         'RESTRICTED_USE': row['RESTRICTED_USE'],
#         'SITE_MGMT_REQ': row['SITE_MGMT_REQ'],
#         'FUNDING': row['FUNDING'],
#         'case_details': row['case_details']
#     }

#     # Replace all NaT values with None for each attribute
#     for key, value in attributes.items():
#         if pd.isna(value):
#             attributes[key] = None

#     feature_dict = {
#         "attributes": attributes,
#         "geometry": {
#             "x": x,
#             "y": y
#         }
#     }
#     new_features.append(feature_dict)

In [ ]:
#new_features

[{'attributes': {'PROJECT_NAME': 'HARRIS DRY CLEANERS',
   'ADDRESS': '2801 MARTIN LUTHER KING JR. WAY',
   'CITY': 'OAKLAND',
   'STATE': 'CA',
   'ZIP': '94609',
   'COUNTY': 'ALAMEDA',
   'SITE_TYPE': 'State Response',
   'SITE_TYPE_DETAILED': 'State Response or NPL',
   'REGULATORY_AGENCIES_INVOLVED': 'SMBRP, RWQCB 2 - San Francisco Bay, US EPA',
   'ENVIROSTOR_ID': '1720109',
   'STATUS': 'Active',
   'STATUS_DATE': '2000-06-16 00:00:00',
   'PAST_USES': 'DRY CLEANING, NONE, NONE, NONE, NONE, NONE',
   'POTENTIAL_MEDIA_AFFECTED': 'OTH, SOIL, SV, IA, IA, IA, IA, IA',
   'RESTRICTED_USE': 'NO',
   'SITE_MGMT_REQ': 'NONE SPECIFIED',
   'FUNDING': 'Orphan Funds',
   'case_details': 'https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=1720109'},
  'geometry': {'x': -13611219.591501093, 'y': 4553765.28337522}},
 {'attributes': {'PROJECT_NAME': 'LOCKHEED PROPULSION-BEAUMONT NO. 2',
   'ADDRESS': 'JACK RABBIT TRAIL',
   'CITY': 'BEAUMONT',
   'STATE': 'CA',
   'ZIP': '92223

In [ ]:
# # determine how many rows we need to delete
# loi = gis.content.get('49bb85274e434c5396daeff04379f315') # envirostor clean up sites id

# old_loi_features = loi.layers[0].query().features

# old_max_loi_objid = max(f.get_value('ObjectId') for f in old_loi_features)

# old_max_loi_objid

14113

In [ ]:
# retry_attempts = 5
# for attempt in range(retry_attempts):
#     try:
#         # Delete data from current
#         deletion_objectids = list(range(1, old_max_loi_objid + 1))

#         # Convert the list of ObjectIds to a comma-separated string
#         deletion_objectids_str = ','.join(map(str, deletion_objectids))

#         # Delete the features by ObjectIds
#         delete_result = loi.layers[0].edit_features(deletes=deletion_objectids_str)
#         break
#     except Exception as e:
#         if "Error Code: 503" in str(e):
#             if attempt < retry_attempts - 1:
#                 print(f"Attempt {attempt + 1} failed. Retrying...")
#                 time.sleep(2 ** attempt)  # exponential backoff
#             else:
#                 print("Max retry attempts reached. Could not complete the request.")
#         else:
#             raise

In [ ]:
# retry_attempts = 10
# for attempt in range(retry_attempts):
#     try:
#         for item in new_features:
#             add_result = loi.layers[0].edit_features(adds=[item])
#         break  # If successful, exit the loop
#     except Exception as e:
#         if "Error Code: 503" in str(e):
#             if attempt < retry_attempts - 1:
#                 print(f"Attempt {attempt + 1} failed. Retrying...")
#                 time.sleep(2 ** attempt)  # Exponential backoff
#             else:
#                 print("Max retry attempts reached. Could not complete the request.")
#         else:
#             raise  # Re-raise the exception if it's not a 503 error

Exception: An error occurred.
(Error Code: 503)

In [ ]:
# # ensure the number of rows match or else throw an error
# if envirostor_cs_sites_clean.shape[0] != loi.layers[0].query().sdf.shape[0]:
#     raise ValueError("The uploaded layer and the input file do not have the same number of rows.")
# else:
#     print("The uploaded layer has the same number of rows as the input file.")

## EnviroStor: Permitted Sites

In [7]:
# envirostor: permitted sites
getOnlineFile(url_of_interest = "https://www.envirostor.dtsc.ca.gov/public/hwmpexport/EnviroStorHazardousWaste.zip", file_of_interest_name = "hazardous_waste_facilities", file_of_interest_type = ".txt", file_name_prefix = "envirostor_hw")

envirostor_hw_hazardous_waste_facilities_linked = envirostor_hw_hazardous_waste_facilities
envirostor_hw_hazardous_waste_facilities_linked['EPA_ID'] = envirostor_hw_hazardous_waste_facilities_linked['EPA_ID'].astype(str)
envirostor_hw_hazardous_waste_facilities_linked['case_details'] = "https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=" + envirostor_hw_hazardous_waste_facilities["EPA_ID"]
envirostor_hw_hazardous_waste_facilities_linked.head()
envirostor_hw_hazardous_waste_facilities_linked_nonas = envirostor_hw_hazardous_waste_facilities_linked.dropna(subset=['LATITUDE', 'LONGITUDE'])
print(envirostor_hw_hazardous_waste_facilities_linked.shape)
print(envirostor_hw_hazardous_waste_facilities_linked_nonas.shape)
envirostor_hw_hazardous_waste_facilities_linked_nonas_gdf = gpd.GeoDataFrame(
    envirostor_hw_hazardous_waste_facilities_linked_nonas,
    geometry=[Point(xy) for xy in zip(envirostor_hw_hazardous_waste_facilities_linked_nonas['LONGITUDE'], envirostor_hw_hazardous_waste_facilities_linked_nonas['LATITUDE'])],
    crs="EPSG:4326"
)
envirostor_hw_hazardous_waste_facilities_linked_nonas_gdf.head()
envirostor_hw_hazardous_waste_facilities_clean = envirostor_hw_hazardous_waste_facilities_linked_nonas_gdf[['EPA_ID', 'FACILITY_NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'COUNTY', 'FACILITY_TYPE', 'FACILITY_STATUS', 'COMMERCIAL_OFFSITE_FACILITY_TYPES', 'QUARTERLY_UPDATE', 'PERMIT_TYPE', 'PERMIT_EFFECTIVE_DATE', 'PERMIT_EXPIRATION_DATE', 'case_details', 'geometry']]
print(envirostor_hw_hazardous_waste_facilities_clean.shape)
envirostor_hw_hazardous_waste_facilities_clean.head()

HTTP Status Code: 200
(904, 30)
(904, 30)
(904, 16)


,EPA_ID,FACILITY_NAME,ADDRESS,CITY,STATE,ZIP,COUNTY,FACILITY_TYPE,FACILITY_STATUS,COMMERCIAL_OFFSITE_FACILITY_TYPES,QUARTERLY_UPDATE,PERMIT_TYPE,PERMIT_EFFECTIVE_DATE,PERMIT_EXPIRATION_DATE,case_details,geometry
0,CA0000084517,"SAFETY-KLEEN SYSTEMS,INC.",6000 88TH ST,SACRAMENTO,CA,958280000,SACRAMENTO,Permitted - Operating,OPERATING PERMIT,"Solvent Recovery, Used Oil / Antifreeze","QUARTERLY UPDATE Quarterly Update Period: January 2024 - March 2024 Permit Renewal: On June 27, 2018, the Department of Toxic Substances Control (DTSC) issued a RCRA Hazardous Waste Permit (Permit) to Safety-Kleen Systems Inc. The expiration date for the Permit is July 30, 2028. FACILITY DESCRIPTION/HISTORY: Safety-Kleen, Sacramento Accumulation Center (Facility) is located ...",RCRA,"07/30/2018 (OP),","07/30/2028 (OP),",https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=CA0000084517,POINT (-121.37675 38.51695)
1,CA0000588350,SAFETY-KLEEN SYSTEMS INC,2875 S RESERVOIR ST,POMONA,CA,917666526,LOS ANGELES,Historical - Non-Operating,KNOWN GENERATORS,NaN,NaN,NaN,NaN,NaN,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=CA0000588350,POINT (-117.73161 34.02126)
2,CA0170024491,NAVAL WEAPONS STATION SEAL BEACH,800 SEAL BEACH BLVD CODE 045,SEAL BEACH,CA,907405000,ORANGE,Historical - Non-Operating,CLOSED,NaN,"This is an active facility which is undergoing Corrective Action overseen by the Office of Military Facilities. The facility is currently a hazardous waste generator. The facility clean closed its hazardous waste storage unit (Bldg 38) and two 9,100-gallon-above ground storage tanks in 2002. The closure was acknowledged by DTSC on 5/7/02. As, a result, a hazardous waste facility permit is not ...",RCRA,NaN,NaN,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=CA0170024491,POINT (-118.09435 33.74236)
3,CA0210020780,SACRAMENTO ARMY DEPOT,8350 FRUITRIDGE RD,SACRAMENTO,CA,958280949,SACRAMENTO,Historical - Non-Operating,CLOSED,NaN,"Update 6/19/23: Sacramento Army Depot had one unit that was RCRA permitted. A RCRA permit was issued for container storage unit located in building 412 (City of Sacramento Parcel/Area 2a) on 4/17/85. This permit was renewed on 8/15/92. The closure plan of the building 412 container storage unit was approved on 6/9/95. However, the closure of the unit was referred to DTSC Cleanup Program. The c...",RCRA,NaN,NaN,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=CA0210020780,POINT (-121.39577 38.52519)
4,CA1170090012,UWS NAVY/FLEET & INDUST SUPL CTR-ALAMEDA,2155 MARINER SQUARE LOOP,ALAMEDA,CA,945011022,ALAMEDA,Historical - Non-Operating,CLOSED,NaN,NaN,NaN,NaN,NaN,https://www.envirostor.dtsc.ca.gov/public/profile_report?global_id=CA1170090012,POINT (-122.27974 37.78789)


In [8]:
# get data in correct format
transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

new_features = []

for index, row in envirostor_hw_hazardous_waste_facilities_clean.iterrows():
    # Convert the coordinates
    x, y = transformer.transform(row.geometry.x, row.geometry.y)
    attributes = {
        'EPA_ID': row['EPA_ID'],
        'FACILITY_NAME': row['FACILITY_NAME'],
        'ADDRESS': row['ADDRESS'],
        'CITY': row['CITY'],
        'STATE': row['STATE'],
        'ZIP': row['ZIP'],
        'COUNTY': row['COUNTY'],
        'FACILITY_TYPE': row['FACILITY_TYPE'],
        'FACILITY_STATUS': row['FACILITY_STATUS'],
        'COMMERCIAL_OFFSITE_FACILITY_TYP': row['COMMERCIAL_OFFSITE_FACILITY_TYPES'],
        'QUARTERLY_UPDATE': row['QUARTERLY_UPDATE'],
        'PERMIT_TYPE': row['PERMIT_TYPE'],
        'PERMIT_EFFECTIVE_DATE': row['PERMIT_EFFECTIVE_DATE'],
        'PERMIT_EXPIRATION_DATE': row['PERMIT_EXPIRATION_DATE'],
        'case_details': row['case_details']
    }

    # Replace all NaT values with None for each attribute
    for key, value in attributes.items():
        if pd.isna(value):
            attributes[key] = None

    feature_dict = {
        "attributes": attributes,
        "geometry": {
            "x": x,
            "y": y
        }
    }
    new_features.append(feature_dict)

In [9]:
# determine how many rows we need to delete
loi = gis.content.get('612b2ed5279a4b348da09242ca5d36b2') # envirostor permitted sites id

old_loi_features = loi.layers[0].query().features

old_max_loi_objid = max(f.get_value('ObjectId') for f in old_loi_features)

old_max_loi_objid

904

In [13]:
retry_attempts = 5
for attempt in range(retry_attempts):
    try:
        # Delete data from current
        deletion_objectids = list(range(1, old_max_loi_objid + 1))

        # Convert the list of ObjectIds to a comma-separated string
        deletion_objectids_str = ','.join(map(str, deletion_objectids))

        # Delete the features by ObjectIds
        delete_result = loi.layers[0].edit_features(deletes=deletion_objectids_str)
        break
    except Exception as e:
        if "Error Code: 503" in str(e):
            if attempt < retry_attempts - 1:
                print(f"Attempt {attempt + 1} failed. Retrying...")
                time.sleep(2 ** attempt)  # exponential backoff
            else:
                print("Max retry attempts reached. Could not complete the request.")
        else:
            raise

In [14]:
retry_attempts = 10
for attempt in range(retry_attempts):
    try:
        for item in new_features:
            add_result = loi.layers[0].edit_features(adds=[item])
        break  # If successful, exit the loop
    except Exception as e:
        if "Error Code: 503" in str(e):
            if attempt < retry_attempts - 1:
                print(f"Attempt {attempt + 1} failed. Retrying...")
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                print("Max retry attempts reached. Could not complete the request.")
        else:
            raise  # Re-raise the exception if it's not a 503 error

In [15]:
# ensure the number of rows match or else throw an error
if envirostor_hw_hazardous_waste_facilities_clean.shape[0] != loi.layers[0].query().sdf.shape[0]:
    raise ValueError("The uploaded layer and the input file do not have the same number of rows.")
else:
    print("The uploaded layer has the same number of rows as the input file.")

The uploaded layer has the same number of rows as the input file.


## Geotracker

In [11]:
# getOnlineFile(url_of_interest = "https://geotracker.waterboards.ca.gov/data_download/GeoTrackerDownload.zip", file_of_interest_name = "sites", file_of_interest_type = ".txt", file_name_prefix = "geotracker")

# geotracker_sites_linked = geotracker_sites
# pd.set_option('display.max_colwidth', 400) # 400 characters
# geotracker_sites_linked['case_details'] = "https://geotracker.waterboards.ca.gov/profile_report.asp?global_id=" + geotracker_sites["GLOBAL_ID"]
# geotracker_sites_linked.head()

# # Remove rows with NA in 'lat' or 'lon'
# geotracker_sites_linked_nonas = geotracker_sites_linked.dropna(subset=['LATITUDE', 'LONGITUDE'])

# # Create GeoDataFrame
# geotracker_sites_linked_nonas_gdf = gpd.GeoDataFrame(
#     geotracker_sites_linked_nonas,
#     geometry=[Point(xy) for xy in zip(geotracker_sites_linked_nonas['LONGITUDE'], geotracker_sites_linked_nonas['LATITUDE'])],
#     crs="EPSG:4326"
# )

# # remove extraneous columns: CUF_CASE is clean up fund case - not sure if this is necessary to keep in. RB_CASE_NUMBER is the water board case number
# geotracker_clean = geotracker_sites_linked_nonas_gdf[['GLOBAL_ID', 'BUSINESS_NAME', 'STREET_NUMBER', 'STREET_NAME', 'CITY', 'STATE', 'ZIP','CASE_TYPE', 'STATUS', 'STATUS_DATE', 'POTENTIAL_CONTAMINANTS_OF_CONCERN', 'POTENTIAL_MEDIA_OF_CONCERN', 'SITE_HISTORY', 'DISCHARGE_SOURCE', 'NO_FURTHER_ACTION_DATE', 'CALWATER_WATERSHED_NAME', 'DWR_GROUNDWATER_SUBBASIN_NAME', 'case_details', 'geometry']]
# geotracker_clean.head()

HTTP Status Code: 200


,GLOBAL_ID,BUSINESS_NAME,STREET_NUMBER,STREET_NAME,CITY,STATE,ZIP,CASE_TYPE,STATUS,STATUS_DATE,POTENTIAL_CONTAMINANTS_OF_CONCERN,POTENTIAL_MEDIA_OF_CONCERN,SITE_HISTORY,DISCHARGE_SOURCE,NO_FURTHER_ACTION_DATE,CALWATER_WATERSHED_NAME,DWR_GROUNDWATER_SUBBASIN_NAME,case_details,geometry
0,SL205413012,KMEP Fox Road Petroleum Pipeline Release,NaN,FOX ROAD,Dixon,CA,95620,Cleanup Program Site,Open - Remediation,2001-04-27 00:00:00,"* Petroleum - Automotive gasolines, * Petroleum - Diesel fuels",Aquifer used for drinking water supply,"The Site is about three miles northeast of the town of Elmira and is surrounded by farm land and a few nearby residences and farm buildings. In November 1993, a release of refined petroleum product was identified in a 14-inch diameter petroleum pipeline owned by SFPP that runs parrallel to and within the right-of-way for the Union Pacific Railroad Company railroad tracks near Fox Road in Sola...",NaN,NaN,Valley Putah-Cache - Elmira (511.10),Sacramento Valley - Solano (5-021.66),https://geotracker.waterboards.ca.gov/profile_report.asp?global_id=SL205413012,POINT (-121.88045 38.38268)
1,T0610700204,VISALIA PUBLIC CEMETERY,1300,GOSHEN W,VISALIA,CA,93291,LUST Cleanup Site,Completed - Case Closed,1996-07-02 00:00:00,Gasoline,Soil,NaN,Tank,1996-07-02 00:00:00,South Valley Floor - Kaweah Delta (558.10),San Joaquin Valley - Kaweah (5-022.11),https://geotracker.waterboards.ca.gov/profile_report.asp?global_id=T0610700204,POINT (-119.30490 36.33457)
2,T0607501311,Commercial,721,Geary Street,San Francisco,CA,94108,LUST Cleanup Site,Completed - Case Closed,1999-10-01 00:00:00,Heating Oil / Fuel Oil,Soil,NaN,NaN,1999-10-01 00:00:00,South Bay - San Mateo Bayside (204.40),Downtown (2-040),https://geotracker.waterboards.ca.gov/profile_report.asp?global_id=T0607501311,POINT (-122.41516 37.78619)
3,T0606700954,DARPETRO (FORMER SS),6421,WATT AVE,NORTH HIGHLANDS,CA,95660,LUST Cleanup Site,Completed - Case Closed,1997-05-20 00:00:00,Gasoline,Soil,NaN,NaN,1997-05-20 00:00:00,Valley-American - Coon-American - Lower American (519.21),Sacramento Valley - North American (5-021.64),https://geotracker.waterboards.ca.gov/profile_report.asp?global_id=T0606700954,POINT (-121.38361 38.68337)
4,T0609700892,Hooker's Texaco (former),16820,Sonoma Hwy,Sonoma,CA,95476,LUST Cleanup Site,Completed - Case Closed,1999-07-19 00:00:00,"Gasoline, Diesel",Aquifer used for drinking water supply,NaN,NaN,1999-07-19 00:00:00,San Pablo - Sonoma Creek (206.40),Napa-Sonoma Valley - Sonoma Valley (2-002.02),https://geotracker.waterboards.ca.gov/profile_report.asp?global_id=T0609700892,POINT (-122.48995 38.32851)


In [ ]:
# # get data in correct format
# transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

# new_features = []

# for index, row in geotracker_clean.iterrows():
#     # Convert the coordinates
#     x, y = transformer.transform(row.geometry.x, row.geometry.y)
#     attributes = {
#         'GLOBAL_ID': row['GLOBAL_ID'],
#         'BUSINESS_NAME': row['BUSINESS_NAME'],
#         'STREET_NUMBER': row['STREET_NUMBER'],
#         'STREET_NAME': row['STREET_NAME'],
#         'CITY': row['CITY'],
#         'STATE': row['STATE'],
#         'ZIP': row['ZIP'],
#         'CASE_TYPE': row['CASE_TYPE'],
#         'STATUS': row['STATUS'],
#         'STATUS_DATE': row['STATUS_DATE'],
#         'POTENTIAL_CONTAMINANTS_OF_CONCE': row['POTENTIAL_CONTAMINANTS_OF_CONCERN'],
#         'POTENTIAL_MEDIA_OF_CONCERN': row['POTENTIAL_MEDIA_OF_CONCERN'],
#         'SITE_HISTORY': row['SITE_HISTORY'],
#         'DISCHARGE_SOURCE': row['DISCHARGE_SOURCE'],
#         'NO_FURTHER_ACTION_DATE': row['NO_FURTHER_ACTION_DATE'],
#         'CALWATER_WATERSHED_NAME': row['CALWATER_WATERSHED_NAME'],
#         'DWR_GROUNDWATER_SUBBASIN_NAME': row['DWR_GROUNDWATER_SUBBASIN_NAME'],
#         'case_details': row['case_details']
#     }

#     # Replace all NaT values with None for each attribute
#     for key, value in attributes.items():
#         if pd.isna(value):
#             attributes[key] = None

#     feature_dict = {
#         "attributes": attributes,
#         "geometry": {
#             "x": x,
#             "y": y
#         }
#     }
#     new_features.append(feature_dict)

In [10]:
# # determine how many rows we need to delete
# loi = gis.content.get('0df71f17a1d54b4a99f814558fe5ca52') # geotracker

# old_loi_features = loi.layers[0].query().features

# old_max_loi_objid = max(f.get_value('ObjectId') for f in old_loi_features)

# old_max_loi_objid

NameError: name 'gis' is not defined

In [ ]:
# retry_attempts = 5
# for attempt in range(retry_attempts):
#     try:
#         # Delete data from current
#         deletion_objectids = list(range(1, old_max_loi_objid + 1))

#         # Convert the list of ObjectIds to a comma-separated string
#         deletion_objectids_str = ','.join(map(str, deletion_objectids))

#         # Delete the features by ObjectIds
#         delete_result = loi.layers[0].edit_features(deletes=deletion_objectids_str)
#         break
#     except Exception as e:
#         if "Error Code: 503" in str(e):
#             if attempt < retry_attempts - 1:
#                 print(f"Attempt {attempt + 1} failed. Retrying...")
#                 time.sleep(2 ** attempt)  # exponential backoff
#             else:
#                 print("Max retry attempts reached. Could not complete the request.")
#         else:
#             raise

In [12]:
# geotracker_clean.shape

(74180, 19)

In [ ]:
# retry_attempts = 70
# for attempt in range(retry_attempts):
#     try:
#         for item in new_features:
#             add_result = loi.layers[0].edit_features(adds=[item])
#         break  # If successful, exit the loop
#     except Exception as e:
#         if "Error Code: 503" in str(e):
#             if attempt < retry_attempts - 1:
#                 print(f"Attempt {attempt + 1} failed. Retrying...")
#                 time.sleep(2 ** attempt)  # Exponential backoff
#             else:
#                 print("Max retry attempts reached. Could not complete the request.")
#         else:
#             raise  # Re-raise the exception if it's not a 503 error

In [ ]:
# # ensure the number of rows match or else throw an error
# if geotracker_clean.shape[0] != loi.layers[0].query().sdf.shape[0]:
#     raise ValueError("The uploaded layer and the input file do not have the same number of rows.")
# else:
#     print("The uploaded layer has the same number of rows as the input file.")

In [13]:
#fused.run("ebe3763fe3c1050257f8d98af26474eba38190602f2c76c75661bd1bcd52183d", x=357, y=817, z=11, engine='realtime')

## Future Work

from the original uploading:
1. Explore how to read in the missing lines that we are currently excluding due to delimiter issues leading to mismatching column numbers for certain rows
2. Confirm with Mark that he doesn't check any additional geotracker layers / all the "types" we have are all he would care about (same thing for envirostor)
3. Email regarding the missing lat/lons (found in geotracker, not sure yet as of 4/17 for envirostor) or geocode those separately